In [118]:
import numpy as np
import pandas as pd
from ckiptagger import WS
from sklearn.model_selection import train_test_split
import json

In [14]:
#斷詞
ws = WS("./ckip")

In [23]:
with open('data/news.json', encoding="utf-8") as f:
    data = json.load(f)

In [28]:
df_data = pd.DataFrame([data[i] for i in range(len(data))])
print("shape of data = {}".format(df.shape))
df_data.head()

shape of data = (7507, 4)


,title,date,author,content
0,《新聞透視》花媽心內話 攪動初選排序,2017/12/26,【李義、林宏聰】,都是心內話，藍綠兩樣情！民進黨5立委搶當高雄市長陳菊接班人，讀著《花媽心內話》各自畫重點...
1,上任後第一把火 韓國瑜議會督軍 藍營炮火猛,2017/09/29,【林宏聰／高雄報導】,「這一小步，代表未來高雄政黨政治一大步！」新任國民黨高市黨部主委韓國瑜，28日首度進市議...
2,價格崩跌 買蕉贈弱勢 挺蕉農 韓國瑜籲政府扛責任,2017/10/15,【林雅惠／高雄報導】,國內香蕉價格自6月起持續下滑，至今產地收購價格每公斤2元到5元，國民黨高市黨部主委韓國瑜...
3,主委坐鎮先禮後兵 藍標語激怒綠,2017/09/29,【林宏聰／高雄報導】,藍綠28日在議會先禮後兵，相互強調理性問政後，國民黨團隨即聯合質詢，重砲抨擊高雄人口變「...
4,不禮讓 大綠議員提名36席起跳,2017/10/25,【李義／高雄報導】,明年大選，民進黨面臨內憂外患，內是新秀爭雄僧多粥少，外有國民黨明星主委韓國瑜領軍要搶攻議...


In [112]:
news_list = ws(df_data["content"])

In [126]:
df_data["content_segmented"] = news_list

In [127]:
df_data.head()

,title,date,author,content,content_segmented
0,《新聞透視》花媽心內話 攪動初選排序,2017/12/26,【李義、林宏聰】,都是心內話，藍綠兩樣情！民進黨5立委搶當高雄市長陳菊接班人，讀著《花媽心內話》各自畫重點...,"[ , 都, 是, 心, 內話, ，, 藍, 綠, 兩樣, 情, ！, 民進黨, 5, 立..."
1,上任後第一把火 韓國瑜議會督軍 藍營炮火猛,2017/09/29,【林宏聰／高雄報導】,「這一小步，代表未來高雄政黨政治一大步！」新任國民黨高市黨部主委韓國瑜，28日首度進市議...,"[ , 「, 這, 一, 小, 步, ，, 代表, 未來, 高雄, 政黨, 政治, 一, ..."
2,價格崩跌 買蕉贈弱勢 挺蕉農 韓國瑜籲政府扛責任,2017/10/15,【林雅惠／高雄報導】,國內香蕉價格自6月起持續下滑，至今產地收購價格每公斤2元到5元，國民黨高市黨部主委韓國瑜...,"[ , 國內, 香蕉, 價格, 自, 6月, 起, 持續, 下滑, ，, 至今, 產地, ..."
3,主委坐鎮先禮後兵 藍標語激怒綠,2017/09/29,【林宏聰／高雄報導】,藍綠28日在議會先禮後兵，相互強調理性問政後，國民黨團隨即聯合質詢，重砲抨擊高雄人口變「...,"[ , 藍綠, 28日, 在, 議會, 先禮, 後兵, ，, 相互, 強調, 理性, 問政..."
4,不禮讓 大綠議員提名36席起跳,2017/10/25,【李義／高雄報導】,明年大選，民進黨面臨內憂外患，內是新秀爭雄僧多粥少，外有國民黨明星主委韓國瑜領軍要搶攻議...,"[ , 明年, 大選, ，, 民進黨, 面臨, 內憂外患, ，, 內, 是, 新秀, 爭雄..."


In [149]:
stopword_punc = []
with open('stop_punctuation.txt', 'r', encoding = 'utf8') as file:
    for data in file.readlines():
        data = data.strip()
        stopword_punc.append(data)

In [150]:
def strQ2B(ustring):
    """把字串全形轉半形"""
    ss = []
    for s in ustring:
        rstring = ""
        for uchar in s:
            inside_code = ord(uchar)
            if inside_code == 12288:  # 全形空格直接轉換
                inside_code = 32
            elif (inside_code >= 65281 and inside_code <= 65374):  # 全形字元（除空格）根據關係轉化
                inside_code -= 65248
            rstring += chr(inside_code)
        ss.append(rstring)
    return ''.join(ss)

In [151]:
def word_preprocessing(news, stopword):
    news = [word.strip(' ') for word in news]
    for word in news:
        if (word == '' or word == '\n'):
            news.remove(word)
        elif word in stopword:
            news.remove(word)
        else:
            word = strQ2B(word)
    return news

In [152]:
df_data["content_segmented"] = df_data["content_segmented"].apply(lambda news : word_preprocessing(news, stopword_punc))

In [153]:
df_data.head()

,title,date,author,content,content_segmented
0,《新聞透視》花媽心內話 攪動初選排序,2017/12/26,【李義、林宏聰】,都是心內話，藍綠兩樣情！民進黨5立委搶當高雄市長陳菊接班人，讀著《花媽心內話》各自畫重點...,"[都, 是, 心, 內話, 藍, 綠, 兩樣, 情, 民進黨, 5, 立委, 搶, 當, 高..."
1,上任後第一把火 韓國瑜議會督軍 藍營炮火猛,2017/09/29,【林宏聰／高雄報導】,「這一小步，代表未來高雄政黨政治一大步！」新任國民黨高市黨部主委韓國瑜，28日首度進市議...,"[這, 一, 小, 步, 代表, 未來, 高雄, 政黨, 政治, 一, 大, 步, 新任, ..."
2,價格崩跌 買蕉贈弱勢 挺蕉農 韓國瑜籲政府扛責任,2017/10/15,【林雅惠／高雄報導】,國內香蕉價格自6月起持續下滑，至今產地收購價格每公斤2元到5元，國民黨高市黨部主委韓國瑜...,"[國內, 香蕉, 價格, 自, 6月, 起, 持續, 下滑, 至今, 產地, 收購, 價格,..."
3,主委坐鎮先禮後兵 藍標語激怒綠,2017/09/29,【林宏聰／高雄報導】,藍綠28日在議會先禮後兵，相互強調理性問政後，國民黨團隨即聯合質詢，重砲抨擊高雄人口變「...,"[藍綠, 28日, 在, 議會, 先禮, 後兵, 相互, 強調, 理性, 問政, 後, 國民..."
4,不禮讓 大綠議員提名36席起跳,2017/10/25,【李義／高雄報導】,明年大選，民進黨面臨內憂外患，內是新秀爭雄僧多粥少，外有國民黨明星主委韓國瑜領軍要搶攻議...,"[明年, 大選, 民進黨, 面臨, 內憂外患, 內, 是, 新秀, 爭雄, 僧多粥少, 外,..."


In [162]:
#train:valid:test = 0.75:0.15:0.15
train_ratio = 0.70
valid_ratio = 0.15
test_ratio = 0.15
data_train, data_test = train_test_split(df_data, test_size = 1 - train_ratio, random_state = 43)
data_valid, data_test = train_test_split(data_test, test_size = test_ratio/(test_ratio + valid_ratio), random_state = 21)

In [165]:
print("shape of training data = {}".format(data_train.shape))
print("shape of valid data = {}".format(data_valid.shape))
print("shape of testing data = {}".format(data_test.shape))

shape of training data = (5254, 5)
shape of valid data = (1126, 5)
shape of testing data = (1127, 5)


In [ ]:
#save to pickle
.to_pickle("./dummy.pkl")
.to_pickle("./dummy.pkl")
.to_pickle("./dummy.pkl")